# 整数构造函数和进制

整数类型（int）的构造函数与进制转换详解

本代码基于Python内置int类特性，实现了十进制到任意进制（2-36）的转换功能

In [1]:
# 1. 基本构造：直接传入整数（十进制）
int(10)  # 结果：10 → 直接创建十进制整数10

10

In [2]:
# 2. 传入浮点数：向零截断（丢弃小数部分）
int(10.9)  # 结果：10 → 正数浮点数截断小数位

10

In [3]:
int(-10.9)  # 结果：-10 → 负数浮点数同样向零截断（非四舍五入）

-10

In [ ]:
# 3. 传入分数（需导入fractions模块）
from fractions import Fraction
a = Fraction(22, 7)
a

Fraction(22, 7)

In [5]:
int(a)

3

In [6]:
# 4. 从字符串创建整数（支持任意进制）
int('10')

10

In [7]:
int('10', 2)

2

In [8]:
int('101', base=2)

5

In [9]:
# 5. 高进制字符串（11-36进制用a-z/A-Z表示数字，不区分大小写）
int('F1A', base=16)

3866

In [10]:
int('f1a', base=16)

3866

In [11]:
int('B1A', 11)

ValueError: invalid literal for int() with base 11: 'B1A'

In [ ]:
# 二、Python内置进制转换函数（bin/oct/hex）

bin(10) # 二进制

'0b1010'

In [ ]:
hex(10) # 十六进制

'0xa'

In [ ]:
oct(10) # 八进制

'0o12'

In [15]:
a = int('1010', 2)
b = int('0b1010', 2)
c = 0b1010
print(a, b, c)

10 10 10


In [16]:
a = int('f1a', 16)
b = int('0xf1a', 16)
c = 0xf1a
print(a, b, c)

3866 3866 3866


In [17]:
a = 0xf1a
b = 0xF1a
c = 0xF1A
print(a, b, c)

3866 3866 3866


In [ ]:
# 三、自定义进制转换函数（十进制→任意进制2-36）

def from_base10(n, b):
    """
    功能：将十进制非负整数n转换为base=b进制的数字列表（返回各位数字的十进制表示）
    参数：
        n: 待转换的十进制整数（必须≥0）
        b: 目标进制（必须≥2）
    异常：
        ValueError: 当b<2或n<0时抛出
    返回：
        list: 进制转换后的数字列表（高位在前，低位在后）
    """
    if b < 2:
        raise ValueError('Base b must be >= 2')  # 进制必须≥2
    if n < 0:
        raise ValueError('Number n must be >= 0')  # 仅支持非负整数
    if n == 0:
        return [0]
    digits = []
    while n > 0:
        n, m = n // b, n % b # n, m = divmod(n, b)
        digits.insert(0, m)
    return digits

In [21]:
from_base10(10, 2)

[1, 0, 1, 0]

In [22]:
from_base10(255, 16)

[15, 15]

In [ ]:
def encode(digits, digit_map):
    """
    功能：将数字列表（from_base10的输出）编码为字符串形式
    参数：
        digits: 数字列表（每个元素是进制中的一位数字，如[15,15]）
        digit_map: 字符映射表（每个索引对应进制中的一个数字，如'0123...ABC...'）
    异常：
        ValueError: 当digit_map长度不足以覆盖digits中的最大数字时抛出
    返回：
        str: 编码后的进制字符串
    """
    if max(digits) >= len(digit_map): # 校验映射表长度：必须能覆盖digits中的所有数字
        raise ValueError("digit_map is not long enough to encode digits")
    encoding = ''.join([digit_map[d] for d in digits])
    return encoding

In [ ]:
encode([1, 0, 1], "FT")

'TFT'

In [25]:
encode([1, 10, 11], "0123456789AB")

'1AB'

In [ ]:
def rebase_from10(number, base):
    """
    功能：整合from_base10和encode，实现十进制整数→任意进制（2-36）字符串的完整转换
    参数：
        number: 待转换的十进制整数（支持正负）
        base: 目标进制（2≤base≤36，因为digit_map仅包含0-9、A-Z共36个字符）
    异常：
        ValueError: 当base不在2-36范围内时抛出
    返回：
        str: 带符号的目标进制字符串（负数前缀为'-'）
    """
    digit_map = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    if base < 2 or base > 36:
        raise ValueError('Invalid base: 2 <= base <= 36')
    sign = -1 if number < 0 else 1
    number *= sign
    digits = from_base10(number, base)
    encoding = encode(digits, digit_map)
    if sign == -1:
        encoding = "-" + encoding
    return encoding

In [27]:
e1 = rebase_from10(10, 2)
print(e1)
print(int(e1, 2))

1010
10


In [ ]:
# 测试二进制
e2 = rebase_from10(-10, 2)
print(e2)
print(int(e2, 2))

-1010
-10


In [ ]:
# 测试十一进制
rebase_from10(131, 11)

'10A'

In [ ]:
# 测试十六进制
rebase_from10(4095, 16)

'FFF'

In [ ]:
# 测试负数
rebase_from10(-4095, 16)

'-FFF'

# 总结

1. int 构造函数核心用法：支持整数 / 浮点数 / 分数 / 字符串入参，字符串可通过base指定进制（2-36），浮点数 / 分数转换时向零截断。
2. 进制字符规则：11-36 进制用 a-z/A-Z 表示数字，字母不区分大小写；带进制前缀（0b/0o/0x）的字符串转换时，base需匹配前缀类型。
3. Python 内置进制函数：bin()/oct()/hex()实现十进制转 2/8/16 进制，返回带前缀字符串；可直接用进制字面量（0b/0o/0x）定义整数。
4. 自定义进制转换核心逻辑：十进制转任意进制（2-36）的本质是循环取整数商和余数，最终调整为高位在前的顺序。
5. 边界与合法性校验：单独处理n=0（任意进制均为 0）；提前校验base≥2、n≥0，非法输入抛出ValueError异常。
6. 负数进制转换处理：先保存符号，将负数转为正数完成进制转换，最后为结果添加负号，保留原数符号属性。
7. 数字编码映射：通过字符映射表（0-9+A-Z），将进制转换后的数字列表，编码为直观的目标进制字符串（如 15→F）。